<a href="https://colab.research.google.com/github/CmilAmaya/intelligent_agents/blob/main/Dream_team_Practica_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3.5
### [Introducción a los sistemas inteligentes](https://fagonzalezo.github.io/)
___________

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/0zhMgxCML3Q8tSG35kbE) antes del final de la clase. El archivo debe nombrarse como  `isi-practica3.5-unalusername1-unalusername2-unalusername3.ipynb`, donde unalusername es el nombre de usuario asignado por la universidad de cada uno de los miembros del grupo.

**Grupo de trabajo:**

* Juan David Rodríguez Gomez juarodriguezgo@unal.edu.co
* María Camila Amaya Rodríguez mamayaro@unal.edu.co
* Sergio Andrés Castro Vargas secastrov@unal.edu.co

___________

## Triple Cross Puzzle

El *Triple Cross* es un rompecabezas diseñado por la empresa Binary Arts.

<img src="https://www.cs.brandeis.edu/~storer/JimPuzzles/SLIDE/PortToPort/TripleCrossPhoto-TN.jpg"
alt="Triple Cross " width="240" height="180" border="10" />

Aquí puede encontrar más detalles: https://www.jaapsch.net/puzzles/port.htm.

Su objetivo es modelar el *Triple Cross* como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.

_________


### 1. Cree una clase para modelar el problema del Triple Cross

Debe utilizar la plantilla de clase que se encuentra en la siguiente celda.

#### Clase TripleCross_problem

In [ ]:
GOAL_STATE = (
    0,0,0,0,0,1,1,0,0,
    0,2,3,4,1,1,5,6,0,
    0,1,1,1,1,1,7,8,0,
    0,0,9,10,0,0,0,0,0
)

In [ ]:
ACTIONS  = ["M-","M+","P-","P+","Q-","Q+"]

In [ ]:
# This is the Problem class from AIMA, you don't have to modify it

class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)


class TripleCross_problem(Problem):

    def __init__(self, initial):
        '''
        Store the initial state in the problem representation and any useful
        data.
        '''
        self.expanded = 0
        self.goal = GOAL_STATE
        super().__init__(initial=tuple(initial), goal=self.goal)
        self.pattern_lines   = {2, 3, 4}
        self.pattern_circles = {5, 6, 7, 8}
        self.pattern_logos   = {9, 10}
        self.relevant_tiles  = self.pattern_lines | self.pattern_circles | self.pattern_logos

        # Diccionario de movimientos
        self.ACTION_AFFECTED = {
            "M": [9,10,11,12,13,14,15,16,17] + [18,19,20,21,22,23,24,25,26],
            "Q": [5,6,14,15,23,24,32,33],
            "P": [2,3,11,12,20,21,29,30]
        }

        # Precalcular "capacidad" máxima de corrección
        relevant_goal_positions = {i for i,t in enumerate(self.goal) if t in self.relevant_tiles}
        self.max_fixable = 1
        for affected in self.ACTION_AFFECTED.values():
            cnt = sum(1 for pos in affected if pos in relevant_goal_positions)
            self.max_fixable = max(self.max_fixable, cnt)

    def actions(self, state):
        """Return a list of actions that can be executed in the given
        state."""

        return ACTIONS

    def result(self, state, action):
        """
        Return the state that results from executing the given
        action at the given state. The action must be one of
        self.actions(state).
        """
        new_state = list(state)  # Create a copy to avoid modifying the original state

        # Your code here
        movements = {
            "M": [9,10,11,12,13,14,15,16,17] + [18,19,20,21,22,23,24,25,26],
            "Q":  [5,6,14,15,23,24,32,33],
            "P":  [2,3,11,12,20,21,29,30]
        }

        piece = action[0]
        direction = action[1]
        affected_positions = movements[piece]

        #M constants (limits)
        M1 = 8
        M2 = len(movements["M"])

        #Not possible to move to left
        first_position_M_1 = 9
        first_position_M_2 = 18

        #Not possible to move to right
        last_position_M_1 = 17
        last_position_M_2 = 26

        if piece == "M":
          ##move 2 rows
          valuesFirstRow = [state[i] for i in affected_positions[:M1+1]]
          valuesSecondRow = [state[i] for i in affected_positions[M1+1:M2+1]]

          if direction == "-":

              #Check if possible to move
              if valuesFirstRow[0] != 0 and valuesSecondRow[0] != 0:
                return tuple(new_state)


              valuesFirstRow = valuesFirstRow[1:] + [valuesFirstRow[0]]
              valuesSecondRow = valuesSecondRow[1:] + [valuesSecondRow[0]]

              values = valuesFirstRow + valuesSecondRow
          else:

              #Check if possible to move
              if valuesFirstRow[-1] != 0 and valuesSecondRow[-1] != 0:
                return tuple(new_state)

              valuesFirstRow = [valuesFirstRow[-1]] + valuesFirstRow[:-1]
              valuesSecondRow = [valuesSecondRow[-1]] + valuesSecondRow[:-1]

              values = valuesFirstRow + valuesSecondRow


        elif piece == "Q" or piece == "P":
          values = [state[i] for i in affected_positions]
          if direction == "-":
            #check if possible to move
            if(values[-1] != 0 and values[-2] != 0):

              return tuple(new_state)

            values = values[-2:] + values[:-2]
          else:
            #check if possible to move
            if(values[0] != 0 and values[1] != 0):

              return tuple(new_state)
            values = values[2:] + values[:2]

        else:
          pass


        #update state
        for pos, val in zip(affected_positions, values):
            new_state[pos] = val

        return tuple(new_state)  # Return the new state as a tuple

    def is_goal(self, state):
      return state == self.goal


    def action_cost(self, s, a, s1):
        """
        Return the cost of a solution path that arrives at s1 from
        state s via action a.
        """
        return 1  # Uniform cost for all actions

    #Selector de heurística
    def h(self, node):
      return h2(self, node)

    def h2(self, node):
      """The Manhattan heuristic."""
      X = (0, 1, 2, 3,
           0, 1, 2, 3,
           0, 1, 2, 3,
           0, 1, 2, 3)

      Y = (0, 0, 0, 0,
           1, 1, 1, 1,
           2, 2, 2, 2,
           3, 3, 3, 3)
      return sum(abs(X[s-1] - X[g-1]) + abs(Y[s-1] - Y[g-1])
                  for (s, g) in zip(node.state, self.goal))

    def h3(self, node):
      """Heurística #3: capacity.
      Cuenta cuántas fichas relevantes están fuera de lugar
      y divide entre la máxima cantidad que un movimiento podría arreglar."""
      misplaced = sum(
          1 for i, t in enumerate(node.state)
          if t in self.relevant_tiles and t != self.goal[i]
      )
      import math
      return math.ceil(misplaced / self.max_fixable)

    def h4(self, node):
      """Heurística #4: basada en grupos (línea, círculo, logos)."""
      s = node.state
      goal = self.goal
      mis_lines = sum(1 for i,t in enumerate(s)
                      if t in self.pattern_lines and goal[i] not in self.pattern_lines)
      mis_circles = sum(1 for i,t in enumerate(s)
                        if t in self.pattern_circles and goal[i] not in self.pattern_circles)
      mis_logos = sum(1 for i,t in enumerate(s)
                      if t in self.pattern_logos and goal[i] not in self.pattern_logos)
      return mis_lines + mis_circles + mis_logos



def board36(board, fmt=(4 * '{} {} {} {} {} {} {} {} {}\n')):
  "A string representing an 36-puzzle board"
  return fmt.format(*board)



In [ ]:
problem = TripleCross_problem(GOAL_STATE)
state = problem.initial
print(board36(state))

state = problem.result(state, "M-")
print(board36(state))
state = problem.result(state, "P+")
print(board36(state))
state = problem.result(state, "Q-")
print(board36(state))



0 0 0 0 0 1 1 0 0
0 2 3 4 1 1 5 6 0
0 1 1 1 1 1 7 8 0
0 0 9 10 0 0 0 0 0

0 0 0 0 0 1 1 0 0
2 3 4 1 1 5 6 0 0
1 1 1 1 1 7 8 0 0
0 0 9 10 0 0 0 0 0

0 0 4 1 0 1 1 0 0
2 3 1 1 1 5 6 0 0
1 1 9 10 1 7 8 0 0
0 0 0 0 0 0 0 0 0

0 0 4 1 0 0 0 0 0
2 3 1 1 1 1 1 0 0
1 1 9 10 1 5 6 0 0
0 0 0 0 0 7 8 0 0



In [ ]:
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

failure = object()
cutoff  = object()

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost

def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)

    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state

            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result


def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    N = 0
    while frontier:
        node = frontier.pop()

        if problem.is_goal(node.state):
            return node,N

        elif len(node) >= limit:
            result = cutoff

        elif not is_cycle(node):
            N+=1
            for child in expand(problem, node):
                frontier.append(child)
    return result


def g(n): return n.path_cost


def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    N = 0
    while frontier:
        node = frontier.pop()
        N+=1
        if problem.is_goal(node.state):

            return node, N
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))



### 2. Evalue su código con diferentes estrategias de búsqueda



Consulte el código en el repositorio de AIMA (https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) y utilice las implementaciones de búsqueda en amplitud y búsqueda en profundidad iterativa.

Pruebe varias configuraciones iniciales y pruebe cuál es la profundidad máxima que los métodos no informados pueden manejar.

In [ ]:
def bfs(problem):
    """
    Perform a breadth-first search on the problem.
    Return the list of actions to reach the goal state.
    """
    states = []
    result = breadth_first_search(problem)
    actions = path_actions(result)
    print(actions, "\n")

    for s in path_states(result):
        print(board36(s))
        states.append(s)

def iterativeDeepeningSearch(problem):
    """
    Perform an iterative deepening search on the problem.
    Return the list of actions to reach the goal state.
    """
    result = iterative_deepening_search(problem)
    actions = path_actions(result[0])
    states = path_states(result[0])
    print(actions, "\n")

    for s in states:
      print(board36(s))

bfs(problem)
iterativeDeepeningSearch(problem)

[] 

0 0 0 0 0 1 1 0 0
0 2 3 4 1 1 5 6 0
0 1 1 1 1 1 7 8 0
0 0 9 10 0 0 0 0 0

[] 

0 0 0 0 0 1 1 0 0
0 2 3 4 1 1 5 6 0
0 1 1 1 1 1 7 8 0
0 0 9 10 0 0 0 0 0



In [ ]:
problem = TripleCross_problem(state)

print("Breadth first search\n")
actions_bfs = bfs(problem)
print("Iterative deepening search\n")
actions_ids = iterativeDeepeningSearch(problem)

Breadth first search

['P-', 'Q+', 'M+'] 

0 0 4 1 0 0 0 0 0
2 3 1 1 1 1 1 0 0
1 1 9 10 1 5 6 0 0
0 0 0 0 0 7 8 0 0

0 0 0 0 0 0 0 0 0
2 3 4 1 1 1 1 0 0
1 1 1 1 1 5 6 0 0
0 0 9 10 0 7 8 0 0

0 0 0 0 0 1 1 0 0
2 3 4 1 1 5 6 0 0
1 1 1 1 1 7 8 0 0
0 0 9 10 0 0 0 0 0

0 0 0 0 0 1 1 0 0
0 2 3 4 1 1 5 6 0
0 1 1 1 1 1 7 8 0
0 0 9 10 0 0 0 0 0

Iterative deepening search

['Q+', 'P-', 'M+'] 

0 0 4 1 0 0 0 0 0
2 3 1 1 1 1 1 0 0
1 1 9 10 1 5 6 0 0
0 0 0 0 0 7 8 0 0

0 0 4 1 0 1 1 0 0
2 3 1 1 1 5 6 0 0
1 1 9 10 1 7 8 0 0
0 0 0 0 0 0 0 0 0

0 0 0 0 0 1 1 0 0
2 3 4 1 1 5 6 0 0
1 1 1 1 1 7 8 0 0
0 0 9 10 0 0 0 0 0

0 0 0 0 0 1 1 0 0
0 2 3 4 1 1 5 6 0
0 1 1 1 1 1 7 8 0
0 0 9 10 0 0 0 0 0



## 3. Implemente diferente heurísticas para el problema

Consulte el código en el repositorio de AIMA (https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) y utilice la implementación de A*.

Implemente al menos dos heurísticas admisibles y consistentes. Compare A * usando la heurística contra IDS calculando el número de nodos expandidos y el factor de ramificación efectivo, de la misma forma como se hace en la figura 3.29 de la 3ra edición de [Russell10].

In [ ]:
def nullHeuristic(state):
    return 0

def aStarSearch(problem, h=nullHeuristic):
    """
    Perform an A*t search on the problem.
    Returns the list of actions to reach the goal state.
    """
    result = astar_search(problem, h)
    return path_actions(result[0])

def myHeuristic(state):
    ### your code here ###
    return 0



In [ ]:
problem = TripleCross_problem(state)

print("A* WITH H2")
result_h2 = astar_search(problem, problem.h4)
N_h2 = result_h2[1]
depth = len(path_states(result_h2[0])) - 1
print("Nodos expandidos", N_h2)
print("A* (h2) DEPTH", depth)
print("Branching factor", N_h2**(1/depth))

A* WITH H2
Nodos expandidos 6
A* (h2) DEPTH 3
Branching factor 1.8171205928321397
